In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire, milling, conversions, structures

from fibsem.structures import BeamType, Point, FibsemRectangle


In [ ]:
microscope, settings = utils.setup_session(config_path=r"C:\Users\Admin\Github\autolamella\autolamella")

In [ ]:

settings.image.beam_type = BeamType.ION
ref_image = acquire.new_image(microscope, settings.image)


In [ ]:




pixelsize = ref_image.metadata.pixel_size.x
fiducial_centre = Point(-50e-6, -10e-6)
fiducial_length = 30e-6 # microscope_settings.protocol["fiducial"]["length"] 
fiducial_area = calculate_fiducial_area(settings, fiducial_centre, fiducial_length, pixelsize)

print(fiducial_area)

settings.image.reduced_area = fiducial_area
image = acquire.new_image(microscope, settings.image)
print(image.metadata.image_settings.reduced_area)


## Minimap
Position Display

In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire, milling, conversions, structures
from fibsem.structures import FibsemImage, BeamType
import os
from fibsem.imaging import _tile

import matplotlib.pyplot as plt
from autolamella.structures import Experiment

# microscope, settings = utils.setup_session()


In [ ]:

PATH = r"C:\Users\Admin\Github\autolamella\autolamella\log\CHROMATIN-270723"
# EXP_PATH = r"C:\Users\Admin\Github\autolamella\autolamella\log\HANNAH-WAFFLE-26072023\experiment.yaml"
exp = Experiment.load(os.path.join(PATH, "experiment.yaml"))
# image = FibsemImage.load(os.path.join(PATH, "overview-image-ion-auto-gamma.tif"))
eb_image = FibsemImage.load(os.path.join(PATH, "overview-electron.tif"))
# ion_image = FibsemImage.load(os.path.join(PATH, "overview-ion-trench.tif"))

plt.imshow(eb_image.data, cmap="gray")
plt.show()

In [ ]:
from autolamella.tools.data import create_history_dataframe, calculate_statistics_dataframe
import plotly.express as px
import pandas as pd


pd.set_option('display.max_rows', 200)

df_experiment, df_history, beam_shift_info, df_steps, df_stage = calculate_statistics_dataframe(PATH)




In [ ]:
# display(df_stage)

df_stage_filter = df_stage[df_stage["stage"] != "Setup"]
# drop relative moves 
df_stage_filter = df_stage_filter[df_stage_filter["type"] != "relative"]

# drop x=0, y=0
df_stage_filter = df_stage_filter[df_stage_filter["x"] != 0]
df_stage_filter = df_stage_filter[df_stage_filter["y"] != 0]


px.line(df_stage_filter, x="x", y="y", color="stage",symbol="lamella",  title="Stage History")


In [ ]:
# display(df_steps)

fig = px.bar(df_steps, x="lamella", y="duration", color="step", title="Step Duration", barmode="stack", facet_col="stage")

fig.show()
# plot time series with x= step_n and y = timestamp with step  as hover text
df_steps.dropna(inplace=True)
df_steps.duration =df_steps.duration.astype(int)

# convert timestamp to datetime, aus timezone 
df_steps.timestamp = pd.to_datetime(df_steps.timestamp, unit="s")

# convert timestamp to australian timezone
df_steps.timestamp = df_steps.timestamp.dt.tz_localize("UTC").dt.tz_convert("Australia/Sydney")

fig = px.scatter(df_steps, x="step_n", y="timestamp", color="stage", symbol="lamella",
    title="AutoLamella Timeline", 
    hover_name="stage", hover_data=["lamella", "step_n", "step"],)
    # size = "duration", size_max=20)

fig.show()

In [ ]:

fig = px.bar(df_steps, x="step", y="duration", color="step", barmode="group", facet_col="stage")
fig.show()

In [ ]:
fig = px.bar(df_steps, x="step", y="duration", color="step", title="Step Duration", facet_col="stage", facet_row="lamella")

fig.show()
# plot time series with x= step_n and y = timestamp with step  as hover text
df_steps.dropna(inplace=True)
df_steps.duration =df_steps.duration.astype(int)

# convert timestamp to datetime, aus timezone 
df_steps.timestamp = pd.to_datetime(df_steps.timestamp, unit="s")

# convert timestamp to australian timezone
df_steps.timestamp = df_steps.timestamp.dt.tz_localize("UTC").dt.tz_convert("Australia/Sydney")

fig = px.scatter(df_steps, x="step_n", y="timestamp", color="stage", symbol="lamella",
    title="AutoLamella Timeline", 
    hover_name="stage", hover_data=["lamella", "step_n", "step"],)
    # size = "duration", size_max=20)

fig.show()

In [ ]:
# display(df_experiment)


In [ ]:

from autolamella.tools.data import create_history_dataframe
df = create_history_dataframe(exp)
display(df)

import plotly.express as px

fig = px.bar(df, x="petname", y="duration", color="stage", barmode="group", hover_data=df.columns)
fig.show()

In [ ]:

positions = {
    "MillTrench": [],
    "MillUndercut": [],
    "SetupLamella": [],
}
for lamella in exp.positions:
    for state in lamella.history:
        
        if state.stage.name in positions.keys():
            positions[state.stage.name].append(state.microscope_state.absolute_position)
            positions[state.stage.name][-1].name = f"{lamella._petname}"# [{state.stage.name}]"

# pprint(positions)

In [ ]:

key = "MillTrench"
fig = _tile._plot_positions(ion_image, positions[key], show=True)
fig.savefig(os.path.join(PATH, f"overview-positions-{key.lower()}.png"), dpi=300, bbox_inches='tight')

In [ ]:
fig = _tile._plot_positions(eb_image, positions["MillUndercut"], show=True)
fig.savefig(os.path.join(PATH, "overview-positions-mill-undercut.png"), dpi=300, bbox_inches='tight')


In [ ]:
fig = _tile._plot_positions(eb_image, positions["SetupLamella"], show=True)
fig.savefig(os.path.join(PATH, "overview-positions-mill-lamella.png"), dpi=300, bbox_inches='tight')

In [ ]:
_n